In [ ]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from rembg import remove

data_dir = r'E:\DS201.Q11\DS201\Lab_2\data\VinaFood21/train'
preprocessed_dir = r'E:\DS201.Q11\DS201\Lab_2\data\VinaFood21_Preprocessed/train'
os.makedirs(preprocessed_dir, exist_ok=True)

def preprocess_image(img, alpha_fore=10, alpha_back=3, erode_size=5, alpha_matting=True):
    """Tiền xử lý ảnh: Resize, Sharpen, CLAHE, Remove Background, Blur background"""
    
    # Resize
    img = cv.resize(img, (224, 224))
    
    # Sharpen ảnh (Unsharp Masking)
    blur_for_sharp = cv.GaussianBlur(img, (5,5), 0)
    sharp_img = cv.addWeighted(img, 1.5, blur_for_sharp, -0.5, 0)

    # CLAHE
    lab = cv.cvtColor(sharp_img, cv.COLOR_RGB2LAB)
    l, a, b = cv.split(lab)
    clahe = cv.createCLAHE(clipLimit=0.5, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv.merge((l, a, b))
    enhanced_img = cv.cvtColor(lab, cv.COLOR_LAB2RGB)

    # Remove background
    rgba_img = remove(
        enhanced_img,
        alpha_matting=alpha_matting,
        alpha_matting_foreground_threshold=alpha_fore,
        alpha_matting_background_threshold=alpha_back,
        alpha_matting_erode_size=erode_size
    )

    if rgba_img.shape[-1] == 4:
        rgb_img = rgba_img[..., :3]
        alpha = rgba_img[..., 3:] / 255.0
        
        # Blur nền
        blurred_bg = cv.GaussianBlur(rgb_img, (15,15), 0)

        # Ghép foreground + blurred background
        final_img = (rgb_img * alpha + blurred_bg * (1 - alpha)).astype(np.uint8)
    else:
        final_img = rgba_img
    
    return final_img


def process_folder(data_folder, save_folder, show_images=False):
    os.makedirs(save_folder, exist_ok=True)
    
    for folder_name in sorted(os.listdir(data_folder)):
        folder_path = os.path.join(data_folder, folder_name)
        if not os.path.isdir(folder_path):
            continue
        
        target_folder = os.path.join(save_folder, folder_name)
        os.makedirs(target_folder, exist_ok=True)
        
        for filename in tqdm(sorted(os.listdir(folder_path)), desc=f"Processing {folder_name}"):
            if not filename.lower().endswith(('.jpg', '.png')):
                continue
            
            img_path = os.path.join(folder_path, filename)
            save_path = os.path.join(target_folder, os.path.splitext(filename)[0] + '.jpg')
            
            img = cv.imread(img_path)
            rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
            
            processed_img = preprocess_image(rgb_img)
            
            plt.imsave(save_path, processed_img)
            
            if show_images:
                plt.figure(figsize=(12,5))
                plt.subplot(1,2,1)
                plt.imshow(rgb_img)
                plt.title("Original")
                plt.axis('off')
                
                plt.subplot(1,2,2)
                plt.imshow(processed_img)
                plt.title("Foreground sharp + Background blurred")
                plt.axis('off')
                
                plt.tight_layout()
                plt.show()

# Chạy tiền xử lý
process_folder(data_dir, preprocessed_dir, show_images=False)


Processing banh-can:  12%|█▏        | 42/363 [00:51<06:28,  1.21s/it]

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 1.000000e-04
Try decreasing discard_threshold or start with a larger shift



Processing banh-can:  90%|████████▉ | 325/363 [06:40<00:46,  1.23s/it]

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 1.000000e-04
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 1.000000e-03
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shif

Processing banh-can:  92%|█████████▏| 333/363 [06:49<00:37,  1.26s/it]

...

Processing hoanh-thanh-nuoc:  10%|█         | 37/361 [00:43<06:01,  1.12s/it]

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 1.000000e-04
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 1.000000e-03
Try decreasing discard_threshold or start with a larger shift



Processing hoanh-thanh-nuoc:  65%|██████▍   | 233/361 [04:41<02:28,  1.16s/it]

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 0.000000e+00
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 1.000000e-04
Try decreasing discard_threshold or start with a larger shift

PERFORMANCE WARNING:
Thresholded incomplete Cholesky decomposition failed due to insufficient positive-definiteness of matrix A with parameters:
    discard_threshold = 1.000000e-04
    shift = 1.000000e-03
Try decreasing discard_threshold or start with a larger shift



Processing hoanh-thanh-nuoc:  91%|█████████ | 329/361 [06:36<00:38,  1.19s/it]

In [ ]:
# img = cv.imread(path)


In [ ]:
# # OpenCV đọc ảnh theo định dạng BGR (Blue-Green-Red).
# # Matplotlib hiển thị ảnh theo định dạng RGB (Red-Green-Blue).
# # Cần chuyển đổi định dạng này trước khi hiển thị.
# img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
# img_rgb = cv.resize(img_rgb, (224, 224))  # Thay đổi kích thước ảnh nếu cần
# # Hiển thị ảnh bằng Matplotlib
# plt.imshow(img_rgb)
# plt.title("Image") # Thêm tiêu đề (tùy chọn)
# plt.axis('off')    # Tắt trục tọa độ (tùy chọn)
# plt.show()